In [30]:
import pandas as pd
import newspaper
from googlesearch import search
import pdb
import tldextract
from datetime import timedelta


In [112]:
def get_url_bias(url, corpus):
    
    try:
        return corpus.loc[corpus['source_url_processed'].str.contains(tldextract.extract(url).domain)]['bias'].values[0]
    except IndexError:
        return 'center'
    
def get_url_fact(url, corpus):
    try:
        return corpus.loc[corpus['source_url_processed'].str.contains(tldextract.extract(url).domain)]['fact'].values[0]
    except IndexError:
        return 'MIXED'
    
def get_search_query(url):

    article = newspaper.Article(url)

    article.download()
    article.parse()
    article.nlp()

    query             = article.title
    date_before       = article.publish_date + timedelta(days=2)
    date_after        = article.publish_date - timedelta(days=2)

    query_time_before = str(date_before.year) +\
                        '-' + str(date_before.month) +\
                        '-' + str(date_before.day)
            
    query_time_after = str(date_after.year) +\
                        '-' + str(date_after.month) +\
                        '-' + str(date_after.day)
            
            
    query = query + ' before:' + query_time_before + ' after:' + query_time_after
        
    return query
    

    
def get_query_results(search_query):
    
    alt_article_lists = [i for i in search(search_query, num = 10)]
    search_results_df = pd.DataFrame(columns = ['link', 'domain', 'title', 'content'])

    search_results_df['link'] = alt_article_lists
    search_results_df['domain'] = search_results_df['link'].apply(lambda x: tldextract.extract(x).domain\
                                                                             + '.' + tldextract.extract(x).suffix)
    
    return search_results_df



def main():

    corpus        = pd.read_csv('data/corpus.csv')
    url           = 'http://fox13now.com/2013/12/30/new-year-new-laws-obamacare-pot-guns-and-drones/'
    
    url_bias      = get_url_bias(url, corpus)
    url_fact      = get_url_fact(url, corpus)
    
    query         = get_search_query(url)
    query_results_df = get_query_results(query)
    query_results_df = query_results_df.fillna('')


    search_results_df = pd.merge(corpus, query_results_df, left_on = 'source_url_processed', right_on = 'domain')
    search_results_df = search_results_df[~search_results_df['bias'].str.replace('-', ' ').str.contains('right')]
    
    return search_results_df
    #return search_results_df[~search_results_df['bias'].str.replace('-', ' ').str.contains(url_bias)]
    
    
def get_alternative_links(url):
    
    corpus        = pd.read_csv('data/corpus.csv')
    url           = 'http://fox13now.com/2013/12/30/new-year-new-laws-obamacare-pot-guns-and-drones/'
    
    url_bias      = get_url_bias(url, corpus)
    url_fact      = get_url_fact(url, corpus)
    
    query            = get_search_query(url)
    query_results_df = get_query_results(query)
    query_results_df = query_results_df.fillna('')


    search_results_df = pd.merge(corpus, query_results_df, left_on = 'source_url_processed', right_on = 'domain')
    search_results_df = search_results_df[~search_results_df['bias'].str.replace('-', ' ').str.contains('right')]
    
    return search_results_df

In [113]:
main()

HTTPError: HTTP Error 429: Too Many Requests

In [102]:
corpus[corpus['bias'].str.replace('-', ' ').str.contains('right')]

,source_url,source_url_processed,URL,fact,bias
2,http://www.fury.news/,fury.news,http://mediabiasfactcheck.com/fury-news/,LOW,extreme-right
4,http://constitution.com/,constitution.com,http://mediabiasfactcheck.com/the-constitution/,LOW,extreme-right
5,http://freebeacon.com/,freebeacon.com,http://mediabiasfactcheck.com/washington-free-...,MIXED,right
6,http://brexitcentral.com,brexitcentral.com,https://mediabiasfactcheck.com/brexitcentral/,MIXED,right
8,https://patriotpost.us/,patriotpost.us,http://mediabiasfactcheck.com/the-patriot-post/,MIXED,right
...,...,...,...,...,...
1053,https://www.hermancain.com/,hermancain.com,https://mediabiasfactcheck.com/hermancain-com/,MIXED,right
1055,https://www.numbersusa.com/,numbersusa.com,http://mediabiasfactcheck.com/numbers-usa/,MIXED,right
1057,https://www.sbsun.com,sbsun.com,https://mediabiasfactcheck.com/san-bernardino-...,HIGH,right-center
1059,http://allenwestrepublic.com/,allenwestrepublic.com,http://mediabiasfactcheck.com/allen-west-repub...,LOW,extreme-right


In [107]:
search_results_df[search_results_df['bias'].str.replace('-', ' ').str.contains('right')]

,source_url,source_url_processed,URL,fact,bias,link,domain,title,content
0,http://triblive.com/,triblive.com,http://mediabiasfactcheck.com/pittsburgh-tribu...,HIGH,right-center,https://archive.triblive.com/wp-content/themes...,triblive.com,,
2,http://www.dailymail.co.uk/ushome/index.html,dailymail.co.uk,http://mediabiasfactcheck.com/daily-mail/,MIXED,right,https://www.dailymail.co.uk/sitemap-articles-d...,dailymail.co.uk,,
7,http://www.unz.com/,unz.com,https://mediabiasfactcheck.com/the-unz-report/,LOW,extreme-right,http://www.unz.com/sbpdl/portlandia-vs-wire-in...,unz.com,,
